In [1]:
import os, torch, umap
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from dags.utils.utils import decode_zip, extract_headers, decode_body
from transformers import AutoModel, AutoTokenizer

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available()) 
# load_dotenv()
# # SCOPES: Gmail read-only
# SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
# token_path = os.getenv("token_path")
# credentials_path = os.getenv("credentials_path")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

PyTorch version: 2.8.0+cu129
CUDA available: True


In [7]:
path_1 = "data/imp_22-08-2025-03-23.json.gz"
path_2 = "data/unimp_22-08-2025-03-23.json.gz"
decompressed_data_1 = decode_zip(path_1)
decompressed_data_2 = decode_zip(path_2)

df_imp =pd.DataFrame(decompressed_data_1)[["Payload"]]
df_unimp =pd.DataFrame(decompressed_data_2)[["Payload"]]

df_imp["Subject"] = df_imp["Payload"].apply(extract_headers)
df_imp["Body"] = df_imp["Payload"].apply(decode_body)
df_imp["Important"] = 1
df_imp = df_imp.drop(["Payload"], axis=1)

df_unimp["Subject"] = df_unimp["Payload"].apply(extract_headers)
df_unimp["Body"] = df_unimp["Payload"].apply(decode_body)
df_unimp["Important"] = 0
df_unimp = df_unimp.drop(["Payload"], axis=1)

In [4]:
train = pd.concat([df_imp, df_unimp])
train.isnull().sum()

Subject       0
Body         53
Important     0
dtype: int64

In [20]:
train["Body"] = train["Body"].fillna(train["Subject"]*5)

In [ ]:
from dags.utils.utils import get_embeddings

In [22]:
embeddings = get_embeddings(train, model_name)

In [23]:
# # UMAP reduction to 2D
# umap_model = umap.UMAP(n_neighbors=10, min_dist=0.3, random_state=42, n_components=2)
# emb_2d = umap_model.fit_transform(embeddings)

# # Some Overlap is expected
# #High-dimensional separation may not be visible in 2d
# #Can try diff embedding/tokenizer


# # Scatterplot
# plt.figure(figsize=(6, 4))
# scatter = plt.scatter(emb_2d[:, 0], emb_2d[:, 1], c=labels, cmap="Spectral", alpha=0.8)
# plt.xlabel("UMAP-1")
# plt.ylabel("UMAP-2")
# plt.title("Sentence Embeddings with UMAP (colored by label)")
# plt.colorbar(scatter, label="Dependent Variable (0/1)")
# plt.show()

In [24]:
labels = train.loc[:,"Important"].values 

X = embeddings
y = torch.from_numpy(labels)

In [25]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle=True, random_state=0)

In [26]:
X_train.cuda()
y_train.cuda()

tensor([1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 0], device='cuda:0')

In [27]:
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
    "max_depth": 13,
    "objective": "binary:logistic", 
    "eval_metric": "logloss",
    "eta": 0.08,
    "tree_method": "hist",
    "device": "cuda"
}
model = xgb.train(params, dtrain, num_boost_round=1000)

In [28]:
dunlabeled = xgb.DMatrix(X_test)
y_pred_proba = model.predict(dunlabeled)

# Convert probabilities to binary predictions
# Decrease threshold for negating False Negatives
y_pred_binary = (y_pred_proba >= 0.40).astype(int)

In [29]:
from sklearn.metrics import confusion_matrix
cm =confusion_matrix(y_test, y_pred_binary)
print(cm)

[[25  6]
 [ 1 30]]


In [ ]:
sub = ""
body = """
"""

In [54]:
d_2={
    "Subject": [sub],
    "Body":[body]
}
df_2 = pd.DataFrame(d_2)

In [55]:
embeddings_2 =get_embeddings(df_2, model_name)

In [56]:
dunlabeled_2 = xgb.DMatrix(embeddings_2)
y_pred_proba_2 = model.predict(dunlabeled_2)
print(y_pred_proba_2)

[0.71449816]
